<a href="https://colab.research.google.com/github/NguyenDucAnforwork/NLP_training/blob/main/Copy_of_HAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This dataset is fetched from:
https://www.kaggle.com/datasets/jarupula/yahoo-answers-dataset

## I use pretrained model Glove for vector embeddings with 50 dimensions

In [10]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device {device}")

using device cuda


In [ ]:
ls

glove.6B.50d.txt  sample_data/  test.csv  train.csv


In [ ]:
#@title install necessary libraries
!pip install gensim

In [2]:
#@title import libraries + declaring necessary parameters
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import gensim.downloader as api
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import re, nltk
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import csv
import nltk
from torchvision import models
from torchsummary import summary
from bs4 import BeautifulSoup
from torch.utils.data import Dataset, DataLoader

### read data + pretrained model


In [5]:
# convert word to vector embedding from pretrained model
file_path = './glove.6B.200d.txt'

vocab = {}

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split()
        word = parts[0]

        vector = torch.tensor([float(val) if val != '-' else 0 for val in parts[1:]])
        vocab[word] = vector

In [56]:
def read_data(datapath):
    documents, labels = [], []
    with open(datapath, encoding='latin-1') as csv_file:
        reader = csv.reader(csv_file, quotechar='"')
        for idx, line in enumerate(reader):
            if (line[0] == ""):
                break
            text = ""
            for tx in line[1:]:
                feature_cleaned = tx.replace('"', '').replace('\\n', ' ').replace('\\', ' ').replace(',', ' ')
                text += feature_cleaned.lower()
                text += " "
            label = int(line[0]) - 1

            if text:
                documents.append(text)
            labels.append(label)
    return documents, labels

X_train, y_train = read_data("./train.csv")
X_test, y_test = read_data("./test.csv")

train_docs, train_labels = X_train[:7680], y_train[:7680]
valid_docs, valid_labels = X_train[7680:9600], y_train[7680:9600]
test_docs, test_labels = X_test[:992], y_test[:992]

In [41]:
from collections import Counter
counter = Counter(train_labels)
most_common = counter.most_common(2)
print(most_common)

[(6, 1896), (4, 1102)]


### preprocessing

In [57]:
#@title remove unwanted characters
nltk.download('punkt')

def split_into_sentences(document):
    document = re.sub(r"[\[\]']", "", document)
    sentences = re.split(r"[.!?,]", document)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

def preprocess(document):
    sentences = split_into_sentences(document)
    sentence_embeddings = []

    for sentence in sentences:
        sentence_embedding = []
        for word in sentence:
            if word in vocab:
                word_embedding = torch.tensor(vocab[word]).float()  # Load embedding from Glove model
            else:
                word_embedding = torch.randn(200)
            sentence_embedding.append(word_embedding)   # (sent_len, embed_size)

        if sentence_embedding:  # Check if sentence_embedding is not empty
            sentence_embeddings.append(torch.stack(sentence_embedding)) # (doc_len, sent_max_len, embed_size)

    if sentence_embeddings:  # Check if sentence_embeddings is not empty
        padded_document = pad_sequence(sentence_embeddings, batch_first=True).to(device)
    else:
        padded_document = torch.tensor([]).to(device)  # Empty tensor if no valid sentences
    return padded_document

# Assuming train_docs, valid_docs, and test_docs are already defined
train_documents = [preprocess(document) for document in train_docs]
valid_documents = [preprocess(document) for document in valid_docs]
test_documents = [preprocess(document) for document in test_docs]

# Now your documents are on the GPU
print("Train document tensor shapes:", [doc.shape for doc in train_documents])
print("Validation document tensor shapes:", [doc.shape for doc in valid_documents])
print("Test document tensor shapes:", [doc.shape for doc in test_documents])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-57-f7865a1179f7>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  word_embedding = torch.tensor(vocab[word]).float()  # Load embedding from Glove model


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [ ]:
#@title padding + truncating
max_num_sent = config['max_num_sent']
max_sent_len = config['max_sent_len']
embed_size = config['embed_dim']

def padding(documents):
    max_num_sent = config['max_num_sent']
    max_sent_len = config['max_sent_len']
    embed_size = config['embed_dim']
    for i, doc in enumerate(documents):
        num_sent, sent_len, _ = doc.size()
        pad_num_sent = max_num_sent - num_sent
        pad_sent_len = max_sent_len - sent_len

        if pad_num_sent > 0:
            pad = torch.zeros(pad_num_sent, sent_len, embed_size).to(device)
            documents[i] = torch.cat([doc, pad], dim=0)

        else:
            documents[i] = documents[i][:max_num_sent, :, :]
        if pad_sent_len > 0:
            pad = torch.zeros(max_num_sent, pad_sent_len, embed_size).to(device)
            documents[i] = torch.cat([documents[i], pad], dim=1)
        else:
            documents[i] = documents[i][:, :max_sent_len, :]
    return documents

train_documents = padding(train_documents)
valid_documents = padding(valid_documents)
test_documents = padding(test_documents)

In [ ]:
print(valid_documents[0].size())

### dataset + dataloader
- size of a batch of data: (batch_size, max_doc_len, max_sent_len, embed_size) [16, 8, 64, 50]


In [45]:
class TextDataset(Dataset):
    def __init__(self, documents, labels):
        self.documents = [doc.to(device) for doc in documents]  # Move documents to GPU
        self.labels = [torch.tensor(label).to(device) for label in labels]    # Convert labels to tensors and move to GPU
        self.embed_size = 50

    def __len__(self):
        return len(self.documents)

    def __getitem__(self, idx):
        if idx >= 0 and idx < len(self.documents):
            return {'idx': idx, 'document': self.documents[idx], 'label': self.labels[idx]}   # (num_sent, max_sent_len, embed_size)

# Assuming train_documents, train_labels, valid_documents, valid_labels, test_documents, and test_labels are already defined
train_dataset = TextDataset(train_documents, train_labels)
test_dataset = TextDataset(test_documents, test_labels)
valid_dataset = TextDataset(valid_documents, valid_labels)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [46]:
for batch in train_loader:
    sample_batch, sample_label = batch['document'], batch['label']
    print(sample_batch.size(), sample_label.size())
    break

torch.Size([32, 8, 30, 200]) torch.Size([32])


In [47]:
print(sample_batch[:, 0, :, :].shape)

torch.Size([32, 30, 200])


In [48]:
#@title configuration
config = {
    "max_num_sent" : 8,
    "max_sent_len" : 30,
    "embed_dim" : 200,
    "batch_size" : 32,
    "context_size1" : 100,
    "context_size2" : 100,
    "hidden_size1" : 50,
    "hidden_size2" : 50,
    "num_class" : 10
}

### model architecture + model summary


In [26]:
class WordEncoder(nn.Module): # (batch_size, max_sent_len, embed_dim)
    def __init__(self, embed_dim=config['embed_dim'], hidden_size1=config['hidden_size1'], batch_size=config['batch_size']):
        super(WordEncoder, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_size1 = hidden_size1
        self.gru = nn.GRU(embed_dim, hidden_size1, bidirectional=True, batch_first=True)
        self.h_0 = nn.Parameter(torch.zeros(2, batch_size, hidden_size1))

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.gru.named_parameters():
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0.0, std=0.05)


    def forward(self, inputs):   # hidden state = [2, batch_size, hidden_size1] cuz it's bidirectional
        outputs, h_n = self.gru(inputs, self.h_0)
        return outputs   # (batch_size, max_sent_len, 2 * hidden_size1)

In [ ]:
a = WordEncoder()
out1 = a(sample_batch[:, 0, :, :])
print(out1.shape)

torch.Size([32, 30, 100])


In [27]:
class WordAttention(nn.Module):   # input_size = 2 * sent_hidden_size   |   sent_hidden_size = context_size
    def __init__(self, input_size=2*config['hidden_size1'], context_size1=config['context_size1']):
        super(WordAttention, self).__init__()
        self.context_size1 = context_size1
        self.score_weight_matrix = nn.Linear(input_size, context_size1, bias=True)
        self.context_vector = nn.Parameter(torch.randn(context_size1, 1))

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0.0, std=0.05)

    def forward(self, output_tensors):   # (batch_size, max_sent_length, 2*sent_hidden_size). notations are little bit confusing
        u_i = torch.tanh(self.score_weight_matrix(output_tensors))   #  (batch_size, max_sent_length, context_size)
        dot_product = torch.matmul(u_i, self.context_vector)  # (batch_size, max_sent_len)
        a_i = nn.Softmax(dim=1)(dot_product)  # (batch_size, max_sent_len, 1)
        s_i = (a_i * output_tensors).sum(dim=1)  # (batch_size, input_size)
        return s_i

In [ ]:
b = WordAttention()
out2 = b(out1)
print(out2.shape)

torch.Size([32, 100])


In [28]:
class SentenceEncoder(nn.Module):
    def __init__(self, input_size=2*config['hidden_size1'], hidden_size2=config['hidden_size2']):  # (batch_size, max_doc_len, context_size1)
        super(SentenceEncoder, self).__init__()
        self.input_size = input_size
        self.gru = nn.GRU(input_size, hidden_size2, bidirectional=True, batch_first=True)   # (batch_size, max_doc_len, (2 * hidden_size2))
        self.h_0 = nn.Parameter(torch.zeros(2, config['batch_size'], hidden_size2))

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0.0, std=0.05)

    def forward(self, inputs):
        outputs, h_n = self.gru(inputs, self.h_0)
        return outputs   # (batch_size, max_doc_len, 2 * hidden_size2)

In [ ]:
outputs = []
for i in range(sample_batch.shape[1]):
    doc = sample_batch[:, i, :, :]
    out1 = a(doc)
    out2 = b(out1)
    outputs.append(out2)
outputs = torch.stack(outputs).transpose(0,1)
print(outputs.shape)
c = SentenceEncoder()
out3 = c(outputs)
print(out3.shape)

torch.Size([32, 8, 100])
torch.Size([32, 8, 100])


In [29]:
class SentenceAttention(nn.Module):
    def __init__(self, input_size=2*config['hidden_size2'], context_size2=config['context_size2']):
        super(SentenceAttention, self).__init__()
        self.context_size2 = context_size2
        self.score_weight_matrix = nn.Linear(input_size, context_size2, bias=True)
        self.softmax = nn.Softmax(dim=1)
        self.context_vector = nn.Parameter(torch.randn(context_size2, 1))

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0.0, std=0.05)

    def forward(self, encoded_sent):   # (batch_size, max_doc_len, input_size)
        u_i = torch.tanh(self.score_weight_matrix(encoded_sent))   # (batch_size, max_doc_len, hidden_size)
        dot_product = torch.matmul(u_i, self.context_vector)  # (batch_size, max_doc_len)
        a_i = nn.Softmax(dim=1)(dot_product)  # (batch_size, max_doc_len)
        s_i = (a_i * encoded_sent).sum(dim=1) # (batch_size, input_size)
        return s_i

In [ ]:
d = SentenceAttention()
out4 = d(out3)
print(out4.shape)

torch.Size([32, 100])


In [ ]:
print(han())

In [30]:
class DocumentClassification(nn.Module):   # (batch_size, 2 * doc_hidden_units)
    def __init__(self, input_size=2*config['hidden_size2'], num_class=config['num_class']):
        super(DocumentClassification, self).__init__()
        self.linear = nn.Linear(input_size, num_class, bias=True)

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0.0, std=0.05)

    def forward(self, encoded_doc):
      p = self.linear(encoded_doc)   # (batch_size, num_class)
      p = nn.Softmax(dim=1)(p)
      return p

In [ ]:
print(out4.shape)
e = DocumentClassification()
out5 = e(out4)
print(out5.shape)

torch.Size([32, 100])
torch.Size([32, 10])


In [31]:
class HAN(nn.Module):
    def __init__(self, embed_dim=config['embed_dim'], hidden_size1=config['hidden_size1'], hidden_size2=config['hidden_size2'], context_size1=config['context_size1'], context_size2=config['context_size2'], num_class=config['num_class']):
        super(HAN, self).__init__()
        self.word_encoder = WordEncoder(embed_dim, hidden_size1)
        self.word_att = WordAttention(2 * hidden_size1, context_size1)
        self.sentence_encoder = SentenceEncoder(2 * hidden_size1, hidden_size2)
        self.sentence_att = SentenceAttention(2 * hidden_size2, context_size2)
        self.doc_classify = DocumentClassification(2 * hidden_size2, num_class)

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0.0, std=0.05)

    def forward(self, inputs):   # (batch_size, max_doc_len, max_sent_len, embed_size)
        document_embedding = []   # (max_doc_len, batch_size, 2 * hidden_size1)
        for i in range(inputs.shape[1]):
            sentences = inputs[:, i, :, :]
            out1 = self.word_encoder(sentences)   # (batch_size, max_sent_len, 2 * hidden_size1)
            out2 = self.word_att(out1)   # (batch_size, 2 * hidden_size1)
            document_embedding.append(out2.clone())
        document_embedding = torch.stack(document_embedding).transpose(0, 1)   # (batch_size, max_doc_len, 2 * hidden_size1)
        encoded_document = self.sentence_encoder(document_embedding)
        out = self.sentence_att(encoded_document)
        predictions = self.doc_classify(out)
        return predictions

In [ ]:
han = HAN()
print(han(sample_batch).shape)

torch.Size([32, 10])


In [72]:
#@title print out all learnable parameters
def print_learnable_parameters(model):
    print("Learnable parameters in the model:")
    for name, param in model.named_parameters():
        param.requires_grad=True
        if param.requires_grad:
            print(f"{name}: {param.shape}")
        if param.grad is None:
            print(f"No gradient for {name}")

# Print learnable parameters
print_learnable_parameters(han)


Learnable parameters in the model:
word_encoder.h_0: torch.Size([2, 32, 50])
word_encoder.gru.weight_ih_l0: torch.Size([150, 200])
word_encoder.gru.weight_hh_l0: torch.Size([150, 50])
word_encoder.gru.bias_ih_l0: torch.Size([150])
word_encoder.gru.bias_hh_l0: torch.Size([150])
word_encoder.gru.weight_ih_l0_reverse: torch.Size([150, 200])
word_encoder.gru.weight_hh_l0_reverse: torch.Size([150, 50])
word_encoder.gru.bias_ih_l0_reverse: torch.Size([150])
word_encoder.gru.bias_hh_l0_reverse: torch.Size([150])
word_att.context_vector: torch.Size([100, 1])
word_att.score_weight_matrix.weight: torch.Size([100, 100])
word_att.score_weight_matrix.bias: torch.Size([100])
sentence_encoder.h_0: torch.Size([2, 32, 50])
sentence_encoder.gru.weight_ih_l0: torch.Size([150, 100])
sentence_encoder.gru.weight_hh_l0: torch.Size([150, 50])
sentence_encoder.gru.bias_ih_l0: torch.Size([150])
sentence_encoder.gru.bias_hh_l0: torch.Size([150])
sentence_encoder.gru.weight_ih_l0_reverse: torch.Size([150, 100])
s

In [73]:
#@title change the way model's weights are initialized
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

han.apply(initialize_weights)

HAN(
  (word_encoder): WordEncoder(
    (gru): GRU(200, 50, batch_first=True, bidirectional=True)
  )
  (word_att): WordAttention(
    (score_weight_matrix): Linear(in_features=100, out_features=100, bias=True)
  )
  (sentence_encoder): SentenceEncoder(
    (gru): GRU(100, 50, batch_first=True, bidirectional=True)
  )
  (sentence_att): SentenceAttention(
    (score_weight_matrix): Linear(in_features=100, out_features=100, bias=True)
    (softmax): Softmax(dim=1)
  )
  (doc_classify): DocumentClassification(
    (linear): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [84]:
#@title Calculate class weights
class_counts = np.bincount(train_labels)
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
class_weights = class_weights.to(device)

### training loop


In [94]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

# han = HAN().to(device)

# Load checkpoint
checkpoint = torch.load('checkpoint.pth')

# Initialize model
han.load_state_dict(checkpoint['model_state_dict'])

# Assuming HAN model and train_loader are already defined
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(han.parameters(), lr=2e-3, momentum=0.8)

num_epochs = 30
previous_epoch_loss = float('inf')
epoch_losses, validation_losses = [], []

han.train()

for epoch in range(num_epochs):
    total_loss = 0
    with tqdm(train_loader, unit="batch") as tepoch:
        for batch in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}")
            optimizer.zero_grad()
            documents, labels = batch['document'].to(device), batch['label'].to(device)
            outputs = han(documents)
            encoded_labels = []
            for label in labels:
                ex = torch.zeros(10)
                ex[int(label) - 1] = 1
                encoded_labels.append(ex)
            stacked_labels = torch.stack(encoded_labels).to(device)
            loss = criterion(outputs, stacked_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            tepoch.set_postfix(loss=loss.item())

    average_loss = total_loss / len(train_loader)
    epoch_losses.append(average_loss)
    print(f'Training loss: {average_loss}')

    # han.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in valid_loader:
            documents, labels = batch['document'].to(device), batch['label'].to(device)
            outputs = han(documents)
            encoded_labels = []
            for label in labels:
                ex = torch.zeros(10)
                ex[int(label) - 1] = 1
                encoded_labels.append(ex)
            stacked_labels = torch.stack(encoded_labels).to(device)
            loss = criterion(outputs, stacked_labels)
            total_val_loss += loss.item()

    average_val_loss = total_val_loss / len(valid_loader)
    validation_losses.append(average_val_loss)
    print(f'Validation loss: {average_val_loss}')

    torch.save({
    'epoch': num_epochs,
    'model_state_dict': han.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': previous_epoch_loss,
    }, 'checkpoint.pth')

    # if abs(previous_epoch_loss - average_loss) < 1e-5:
    #     print(f'Stopping early at epoch {epoch+1} due to small loss change')
    #     break

    previous_epoch_loss = average_loss

# Plotting the loss function
plt.plot(epoch_losses, label='Training Loss')
plt.plot(validation_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.show()


Epoch 1: 100%|██████████| 240/240 [00:06<00:00, 39.83batch/s, loss=2.24]


Training loss: 2.214276291926702
Validation loss: 2.1991730610529583


Epoch 2:  71%|███████   | 170/240 [00:04<00:01, 36.44batch/s, loss=2.34]


KeyboardInterrupt: 

In [90]:
#@title print the content of checkpoint.pth
import torch

# Load the checkpoint
checkpoint = torch.load('checkpoint.pth')

# Print the keys in the checkpoint
# print("Keys in checkpoint:")
# for key in checkpoint.keys():
#     print(f"{key}: {checkpoint[key]}")

# If you want to inspect the content of the model state dictionary
print("\nModel state dict:")
for name, param in checkpoint['model_state_dict'].items():
    print(f"{name}: {param.shape}")

# If you want to inspect the optimizer state dictionary
# print("\nOptimizer state dict:")
# for name, param in checkpoint['optimizer_state_dict'].items():
#     print(f"{name}: {param}")

# Print other items if needed
if 'epoch' in checkpoint:
    print(f"\nEpoch: {checkpoint['epoch']}")
if 'loss' in checkpoint:
    print(f"Loss: {checkpoint['loss']}")



Model state dict:
word_encoder.h_0: torch.Size([2, 32, 50])
word_encoder.gru.weight_ih_l0: torch.Size([150, 200])
word_encoder.gru.weight_hh_l0: torch.Size([150, 50])
word_encoder.gru.bias_ih_l0: torch.Size([150])
word_encoder.gru.bias_hh_l0: torch.Size([150])
word_encoder.gru.weight_ih_l0_reverse: torch.Size([150, 200])
word_encoder.gru.weight_hh_l0_reverse: torch.Size([150, 50])
word_encoder.gru.bias_ih_l0_reverse: torch.Size([150])
word_encoder.gru.bias_hh_l0_reverse: torch.Size([150])
word_att.context_vector: torch.Size([100, 1])
word_att.score_weight_matrix.weight: torch.Size([100, 100])
word_att.score_weight_matrix.bias: torch.Size([100])
sentence_encoder.h_0: torch.Size([2, 32, 50])
sentence_encoder.gru.weight_ih_l0: torch.Size([150, 100])
sentence_encoder.gru.weight_hh_l0: torch.Size([150, 50])
sentence_encoder.gru.bias_ih_l0: torch.Size([150])
sentence_encoder.gru.bias_hh_l0: torch.Size([150])
sentence_encoder.gru.weight_ih_l0_reverse: torch.Size([150, 100])
sentence_encoder.

### Inference

In [92]:
# Load checkpoint
checkpoint = torch.load('checkpoint.pth')

# Initialize model
han.load_state_dict(checkpoint['model_state_dict'])

# Evaluation code
han.eval()  # Set the model to evaluation mode
all_predictions = []
all_labels = []

with torch.no_grad():  # Disable gradient computation
    for batch in test_loader:
        documents, labels = batch['document'].to(device), batch['label'].to(device)
        outputs = han(documents).to(device)

        # Get the predicted labels
        predicted_labels = torch.argmax(outputs, dim=1)

        all_predictions.extend(predicted_labels.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Here you can add code to evaluate the predictions
# For example, calculating accuracy or other metrics

fixed_all_labels = [int(label)-1 for label in all_labels]
accuracy = sum(1 for x, y in zip(all_predictions, fixed_all_labels) if x == y) / len(all_labels)
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.2651209677419355


In [93]:
print(len([x for x in all_predictions if x != 5]))


0
